In [ ]:
# https://colab.research.google.com/github/ozanpekmezci/capstone/blob/master/prepare-svhn.ipynb
# please make sure that your GPU RAM is lager than 25GB 
PATH = 'VRDL/VRDL_hw2/' #you should key in the path of folder that contain this ipynb file
'''
for example, in my setting, I put this ipynb file folder in 'VRDL/VRDL_HW2' on My Drive
then PATH will be
'VRDL/VRDL_HW2'
'''

from google.colab import drive
drive.mount('/content/gdrive/')

!ls gdrive/MyDrive/
!nvidia-smi

import os
PATH += 'dataset/'
os.chdir('./gdrive/MyDrive/')
os.chdir(PATH)


Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
 01-拍攝檔-347P.rar
'10 27'
'1589095083691 (1).png'
 1589095083691.png
 1.pdf
 20201108_125633.jpg
 20210721_093952.jpg
 3.pdf
 4.pdf
 520.pdf
 614447012.119631.mp4
'Adobe Scan Apr 12, 2021.pdf'
'Adobe Scan Apr 22, 2021_1.jpg'
'Adobe Scan Apr 22, 2021.pdf'
'Adobe Scan Apr 23, 2021 (1).pdf'
'Adobe Scan Apr 23, 2021.pdf'
'Adobe Scan Feb 10, 2021 (1).pdf'
'Adobe Scan Feb 10, 2021 (3).pdf'
'Adobe Scan Feb 10, 2021.pdf'
'Adobe Scan Feb 22, 2021.pdf'
'Adobe Scan Mar 02, 2021.pdf'
'Adobe Scan Mar 09, 2021 (1).pdf'
'Adobe Scan Mar 09, 2021.pdf'
'Adobe Scan Mar 10, 2021.pdf'
'Adobe Scan Mar 30, 2021.pdf'
'Adobe Scan May 01, 2021.pdf'
'Adobe Scan May 05, 2021 (1).pdf'
'Adobe Scan May 05, 2021.pdf'
'Adobe Scan Oct 08, 2021.pdf'
'Adobe Scan Oct 09, 2021.pdf'
'Adobe Scan Sep 24, 2021.pdf'
'Adobe Scan Sep 29, 2021 (1).pdf'
'Adobe Scan Sep 29, 2021.pdf'
 CACS2019
'Chat his

# Prepare SVHN Dataset
This notebook serves to download and preprocess SVHN data, which will be used by other notebooks to classify street numbers.
## Download, Extraction & Preprocess SVHN Data


In [ ]:
# These are all the modules we'll be using later.
import numpy as np
from six.moves import cPickle as pickle
import json
import os
import h5py
import matplotlib.pyplot as plt
import random
from urllib import urlretrieve
from collections import Counter

### Extracting Data

In [ ]:
# !cp 'train.zip' .
# !cp 'test.zip' .
!unzip -q train.zip
!unzip -q test.zip
!mv train train
!mv test test


replace train/1.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: Ａ
error:  invalid response [Ａ]
replace train/1.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace train/1.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
replace test/100000.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
mv: cannot move 'train' to a subdirectory of itself, 'train/train'
mv: cannot move 'test' to a subdirectory of itself, 'test/test'


### Read the dataset
deal with mat file</br>
you cannot use scipy.io

In [ ]:
train_folders = './digitStruct.mat'

In [ ]:
train_dataset = h5py.File(train_folders, "r")

In [ ]:
train_dataset["digitStruct"].items()

[(u'bbox', <HDF5 dataset "bbox": shape (33402, 1), type "|O">),
 (u'name', <HDF5 dataset "name": shape (33402, 1), type "|O">)]

In [ ]:
# The DigitStructFile is just a wrapper around the h5py data.  It basically references 
#    inf:              The input h5 matlab file
#    digitStructName   The h5 ref to all the file names
#    digitStructBbox   The h5 ref to all struc data
class DigitStructFile:
    def __init__(self, inf):
        self.inf = h5py.File(inf, 'r')
        self.digitStructName = self.inf['digitStruct']['name']
        self.digitStructBbox = self.inf['digitStruct']['bbox']

# getName returns the 'name' string for for the n(th) digitStruct. 
    def getName(self,n):
        return ''.join([chr(c[0]) for c in self.inf[self.digitStructName[n][0]].value])

# bboxHelper handles the coding difference when there is exactly one bbox or an array of bbox. 
    def bboxHelper(self,attr):
        if (len(attr) > 1):
            attr = [self.inf[attr.value[j].item()].value[0][0] for j in range(len(attr))]
        else:
            attr = [attr.value[0][0]]
        return attr

# getBbox returns a dict of data for the n(th) bbox. 
    def getBbox(self,n):
        bbox = {}
        bb = self.digitStructBbox[n].item()
        bbox['height'] = self.bboxHelper(self.inf[bb]["height"])
        bbox['label'] = self.bboxHelper(self.inf[bb]["label"])
        bbox['left'] = self.bboxHelper(self.inf[bb]["left"])
        bbox['top'] = self.bboxHelper(self.inf[bb]["top"])
        bbox['width'] = self.bboxHelper(self.inf[bb]["width"])
        return bbox

    def getDigitStructure(self,n):
        s = self.getBbox(n)
        s['name']=self.getName(n)
        return s

# getAllDigitStructure returns all the digitStruct from the input file.     
    def getAllDigitStructure(self):
        return [self.getDigitStructure(i) for i in range(len(self.digitStructName))]

# Return a restructured version of the dataset (one structure by boxed digit).
#
#   Return a list of such dicts :
#      'filename' : filename of the samples
#      'boxes' : list of such dicts (one by digit) :
#          'label' : 1 to 9 corresponding digits. 10 for digit '0' in image.
#          'left', 'top' : position of bounding box
#          'width', 'height' : dimension of bounding box
#
# Note: We may turn this to a generator, if memory issues arise.
    def getAllDigitStructure_ByDigit(self):
        pictDat = self.getAllDigitStructure()
        result = []
        structCnt = 1
        for i in range(len(pictDat)):
            item = { 'filename' : pictDat[i]["name"] }
            figures = []
            for j in range(len(pictDat[i]['height'])):
                figure = {}
                figure['height'] = pictDat[i]['height'][j]
                figure['label']  = pictDat[i]['label'][j]
                figure['left']   = pictDat[i]['left'][j]
                figure['top']    = pictDat[i]['top'][j]
                figure['width']  = pictDat[i]['width'][j]
                figures.append(figure)
            structCnt = structCnt + 1
            item['boxes'] = figures
            result.append(item)
        return result

In [ ]:
print(train_folders)
train = DigitStructFile(train_folders)
train_data = train.getAllDigitStructure_ByDigit()

# test = DigitStructFile(test_folders)
# test_data = test.getAllDigitStructure_ByDigit()

./digitStruct.mat


In [ ]:
train_data[0]
#a dict with two keys:boxes and filename
#boxes has few boxes describing the boxes in the image

{'boxes': [{'height': 219.0,
   'label': 1.0,
   'left': 246.0,
   'top': 77.0,
   'width': 81.0},
  {'height': 219.0, 'label': 9.0, 'left': 323.0, 'top': 81.0, 'width': 96.0}],
 'filename': '1.png'}

In [ ]:

file = 'multi_bbox_info.json'

try:
    f = open(file, 'w')
    save = {
    'train_data': train_data,
    # 'test_data': test_data,
    }
    json.dump(save, f, ensure_ascii=False, indent=4)
    f.close()
except Exception as e:
    print('Unable to save data to', file, ':', e)
    raise

file = 'multi_bbox_info.json'

with open(file, 'r') as f:
    save = json.load(f)
    train_data = save['train_data']
    # test_data = save['test_data']
    del save
    print('Training set', len(train_data))
    # print('Test set', len(test_data))

('Training set', 33402)
